# Módulo 13: Filogenética

## Descripción general

La filogenética es el estudio de las relaciones evolutivas entre entidades biológicas, a menudo especies, individuos o genes (que pueden denominarse taxones). Los principales elementos de la filogenética se resumen en la siguiente figura.

![Intro](images/phylo1.png)

*Tomado de: https://www.ebi.ac.uk/training/online/courses/introduction-to-phylogenetics/what-is-phylogenetics/*


Los árboles filogenéticos basados en datos del genoma completo nos informan de las relaciones entre aislados bacterianos a una escala muy fina. Cuando combinamos esa información de alta resolución sobre las relaciones evolutivas de los aislados con datos geográficos, podemos comprender mejor la distribución actual del patógeno e inferir los procesos epidemiológicos que han actuado sobre la bacteria a lo largo del tiempo. El ejemplo más sencillo sería que una filogenia mostrara que un patógeno está limitado geográficamente (por ejemplo, los aislados de la misma región siempre se agrupan juntos). Esto podría indicar que el patógeno no se está extendiendo rápidamente. Mientras que un patógeno con una filogenia que muestra que los aislados de regiones distantes están probablemente relacionados con los aislados de regiones cercanas, la interpretación es que es probable que el patógeno se propague a través de las fronteras regionales. La referenciación geográfica de los datos genómicos también puede combinarse con información temporal para estudiar el movimiento de los patógenos en el espacio y el tiempo. Esto es más útil cuando se hace en tiempo real y, por tanto, puede ser útil para la detección y el seguimiento de brotes.

En este módulo está dividido en tres partes donde abordaremos: 
1. Construcción de árboles filogenéticos
2. Identificación de regiones de recombinación
3. Agrupación mediante popPUNK.

### Instalar condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### Instalar programas

In [ ]:
# Instalar FastTree
!conda install bioconda::fasttree

In [ ]:
# Instalar snp-sites
!conda config --add channels conda-forge
!conda config --add channels defaults
!conda config --add channels r
!conda config --add channels bioconda
!conda install snp-sites

In [ ]:
# Instalar PopPunk
!conda install poppunk

### Descargar datos

In [ ]:
!wget 

### Extraer el archivo .tar.gz

In [ ]:
!tar xvf 

## Parte 1: Creando un árbol filogenético usando FastTree

### Paso 1: Realice un alineamiento  de sólo SNP utilizando snp-sites

Crear una filogenia a partir de secuencias genómicas completas puede ser un proceso muy lento y de gran carga computacional. Podemos acelerarlo utilizando sólo los sitios variables (SNPs). Sin embargo, debemos ser conscientes de que incluir sólo los sitios variables puede afectar a las estimaciones de la tasa evolutiva realizadas por el software filogenético, por lo que debemos tener en cuenta los sitios que eliminamos en nuestro análisis.

Para ello utilizaremos snp-sites. Puede ver las opciones de snp-sites con el comando:

In [ ]:
# Ejecutar snp-sites
!snp-sites -h

En primer lugar, elimine todos los sitios invariantes y cree una alineación de secuencias múltiples de sólo SNP. Utilizaremos los resultados de las ejecuciones rápidas descritas en la página anterior. Ejecute el comando:

In [ ]:
!snp-sites -o clean.full.SNPs.aln clean.full.aln

La explicación de este comando es la siguiente:

`snp-sites`: es la herramienta/programa

`-o clean.full.SNPs.aln`: especifica el archivo de salida

`clean.full.aln`: especifica el archivo de entrada - que es una salida de snippy

Podemos ver cuántos sitios invariantes se eliminaron (y qué proporción de A, T, G, C tenían) utilizando:

In [ ]:
!snp-sites -C clean.full.aln

Obtendrá la siguiente salida:

### Paso 2: Creación de un árbol filogenético a partir de los SNPs utilizando FastTree

Puede ver las opciones para fasttree de la siguiente manera:

In [ ]:
!FastTree -h

Generaremos un árbol filogenético de máxima verosimilitud utilizando este comando:

In [ ]:
!FastTree -nt -gtr clean.full.SNPs.aln > clean.full.SNPs.aln.tree

La explicación de este comando es la siguiente:

`FastTree`: es la herramienta/programa

`-nt`: especifica que el alineamiento de entrada es de nucleótidos

`gtr`: especifica el modelo evolutivo

`clean.full.SNPs.aln`: alineamiento de entrada

`clean.full.SNPs.aln.tree`: especifica el nombre del árbol de salida

Exploremos la salida de nuestro comando anterior utilizando el comando:

In [ ]:
!ls -lh clean.full.*

Nuestro árbol de máxima verosimilitud se etiqueta clean.full.SNPs.aln.tree. Podemos visualizarlo con figtree o iTOL.

Usted tendrá la salida:

![iTOL](images/itol.png)

O utilizando el paquete ggtree en R:

```
library(ggtree)

mltree <- midpoint.root(read.tree("clean.full.SNPs.aln.treef")) 

ggtree(mltree) + # plot basic tree

geom_tiplab(size=3) + # add tip labels

geom_treescale() + # add scale bar

xlim(0, 0.0005) # set limits so the plot fits nicely on the screen

```


### Paso 3: Interpretar un árbol filogenético

Debemos recordar que el árbol que hemos creado procede de un alineamiento de SNP únicamente.  Recuerde que cuanto más larga es la rama de una cepa, más mutaciones tiene. La cepa de la muestra ERR2667737 y la cepa de referencia son parecidas entre sí, y presentan algunas mutaciones que también se encuentran en la cepa de la muestra ERR2667694. Las cepas de las muestras ERR2667707 y ERR2667708 comparten mutaciones y presentean mutaciones diferentes al resto de cepas.

___

## Parte 2: Identificación de regiones de recombinación usando Gubbins

Muchas bacterias participan en altas tasas de recombinación homóloga. Esto significa que donan y reciben segmentos de ADN entre sí. En el contexto de un árbol filogenético, en el que comparamos regiones similares y disímiles para determinar el parentesco de los aislados, esto puede resultar problemático y dar lugar a longitudes de rama que reflejen recombinaciones en lugar de divergencia. Esto es muy importante en el caso del *Streptococcus pneumoniae*, que es naturalmente competente, lo que significa que puede captar fácilmente ADN.

[Gubbins](https://github.com/nickjcroucher/gubbins/blob/master/docs/gubbins_manual.md) (Genealogies Unbiased By recomBinations In Nucleotide Sequences) es un algoritmo que identifica iterativamente los loci que contienen elevadas densidades de sustituciones de bases, al tiempo que construye una filogenia basada en las mutaciones puntuales putativas fuera de estas regiones. Las simulaciones demuestran que el algoritmo genera reconstrucciones muy precisas bajo modelos realistas de diversificación a corto plazo de las secuencias mediante mutaciones puntuales y recombinación, y puede ejecutarse en alineaciones de muchos cientos de secuencias de genomas bacterianos. Por tanto, no es adecuado para estudiar la recombinación en la diversidad de toda la especie, algo que puede hacerse gen a gen con programas como fastGEAR. En cambio, funciona con muestras de diversidad limitada, que comparten un ancestro común reciente: una cepa o linaje.

El archivo de entrada necesario para Gubbins es un alineamiento FASTA de todo el genoma. Cada secuencia debe tener un identificador único y deben evitarse los caracteres especiales. Las secuencias sólo deben utilizar los caracteres ACGT (bases de ADN),N (base desconocida) o - (hueco en la alineación). Si se va a incluir un árbol de partida, éste deberá tener un formato Newick. La alineación se genera más fácilmente mediante el mapeo de secuencias contra una secuencia de referencia. Esto puede hacerse con el conocido programa de mapeo Snippy.

Puede ver los comandos Gubbins de la siguiente manera:

In [ ]:
!run_gubbins.py -h

Ejecutaremos la herramienta gubbins en un alineamiento completo del genoma y no en un alineamiento SNPs. Ahora vamos a ejecutar este comando:

In [ ]:
!run_gubbins.py --mar -p output clean.full.al

La explicación de este comando es la siguiente:

`run_gubbins.py`: es la herramienta/programa

`--mar`: Esta opción indica a Gubbins que utilice un algoritmo de Monte Carlo con cadena de Markov (MCMC) para inferir la historia evolutiva de los genomas.

`-p output`: Esta opción especifica el nombre del directorio de salida donde se guardarán los resultados del análisis de Gubbins. Como /data dir es el punto de montaje de $PWD de su host debido a -v, verá los archivos en el $PWD de su host

`clean.full.aln`: fichero de entrada

Este comando puede tardar unos minutos en ejecutarse.

Veamos lo que gubbins ha hecho utilizando el comando (`ls -l output.*`)

Una explicación de los archivos de salida:

`output.branch_base_reconstruction.embl`: Reconstrucción de sustitución de bases en formato EMBL

`output.recombination_predictions.embl`: Predicciones de recombinación en formato EMBL.

`salida.predicciones_recombinación.gff`: Predicciones de recombinación en formato GFF

`salida.sitios_polimorfos_filtrados.phylip`: Alineación en formato Phylip de los sitios polimórficos filtrados utilizados para generar la filogenia en la iteración final

`output.final_tree.tre`: este archivo contiene la filogenia final en formato Newick; las longitudes de rama están en mutaciones puntuales

`output.node_labelled.final_tree.tre`: árbol filogenético final en formato Newick pero con etiquetas internas en los nodos; las longitudes de las ramas están en mutaciones puntuales

`output.log`: archivo de registro que especifica el software utilizado en cada paso del análisis, con las citas correspondientes

`output.per_branch_statistics.csv`: informe por rama de las sustituciones de bases dentro y fuera de los eventos de recombinación

`output.summary_of_snp_distribution.vcf`: archivo VCF que resume la distribución de las mutaciones puntuales.

Puedes explorar estos archivos de salida usando el comando `head`. Por ejemplo, "output.recombination_predictions.gff" es un archivo GFF que contiene un registro de cada bloque de recombinación identificado, cuántos SNPs contiene y qué muestras están afectadas.

EJEMPLO

**output.final_tree.tre** es una filogenia sin regiones de recombinación. Lea en el árbol filtrado gubbins y grafíquelo con ggtree o vusualice en Figtree.

In [ ]:
!gubbins.tree <- midpoint.root(read.tree("gubbins.final_tree.tre")) 

ggtree(gubbins.tree) + # plot basic tree

geom_tiplab(size=3) + # add tip labels

geom_treescale() + # add a scale bar

xlim(0, 300) # change the sizing of the plot so it fits nicely on the screen 

FigTree output:


We can also visualise the recombination blocks using a web tool called phandango. Using your browser, navigate to:  https://jameshadfield.github.io/phandango/#/

You will need the following files (drag and drop):

1. output.final_tree.tre
2. output.recombination_predictions.gff
3. PROKKA_03052023.gff (output from Prokka of Reference_sequence_GPSC46.fa)

Phandango should automatically display blocks of recombination in red (ancestral) and blue (specific to a sample)

![rec](images/rec.jpg)



___

## Parte 3: Agrupación mediante popPUNK

[PopPUNK](https://poppunk.readthedocs.io/en/latest/index.html) es una herramienta para agrupar genomas. Nos referimos a los clusters como clusters de k-mer de longitud variable, o VLKCs. Desde el punto de vista biológico, estos grupos suelen representar cepas distintas. Los subgrupos de cepas se denominan linajes.

La siguiente figura muestra un resumen de cómo ejecutar popPUNK

![punk](images/poppunk_flowchart.png)

*Tomado de: https://poppunk.readthedocs.io/en/latest/overview.html*

### Paso 1: Descarga de la base de datos

Como estamos trabajando con *Streptococcus pneumoniae*, descargaremos la base de datos de referencia GPS y la designación GPS que utilizaremos para agrupar nuestro genoma. También puede acceder a los genomas de referencia de otras especies bacterianas desde esta base de datos. Si una especie no está incluida en esta base de datos, se recomienda que cree su propia base de datos.

La base de datos del genoma de referencia GPS de *Streptococcus pneumoniae* se guarda en su directorio como (GPS_v6) y la designación GPS como (GPS_v6_external_clusters.csv)

### Paso 2: Cree un archivo de texto con los detalles de sus muestras

Cree un archivo que enumere los nombres de sus muestras y las rutas a sus datos de secuencia.

In [ ]:
!echo -e "read_17150_4#79\t/data/read_17150_4#79/read_17150_4#79_1.fastq.gz\t/data/read_17150_4#79/read_17150_4#79_2.fastq.gz" > poppunk_input.tsv


Este archivo de texto contiene los nombres de las muestras y sus datos de secuencia. No tiene encabezamiento, está separado por tabuladores y contiene el nombre de la muestra en la primera columna. Las columnas siguientes pueden contener rutas a datos de lectura ensamblados o sin procesar (el tipo se deducirá automáticamente comprobando la presencia de puntuaciones de calidad).

Ejemplo:

``
read_17150_4#79   /data/read_17150_4#79/read_17150_4#79_1.fastq.gz   /data/read_17150_4#79/read_17150_4#79_2.fastq.gz 
``

### Paso 3: Agrupar los genomas

El comando para agrupar sus genomas es el siguiente:

In [ ]:
!poppunk_assign --db GPS_v6 --query /data/poppunk_input.tsv --output poppunk_clusters --external-clustering GPS_v6_external_clusters.csv 

La explicación de este comando es la siguiente:

`poppunk_assign`: es la herramienta/programa/script

`--db GPS_v6`: especifica la base de datos

`--query /data/poppunk_input.tsv`: recibe la entrada del list.txt

`--output poppunk_clusters`: especifica el archivo de salida

`--external-clustering GPS_v6_external_clusters.csv`: directorio que contiene los clusters GPS para las referencias

`--threads 8`: especifica los núcleos virtuales

Al finalizar, se generará una nueva carpeta "poppunk_clusters". Navega hasta esta carpeta y explora su contenido.

Los archivos de salida:

**poppunk_clusters_clusters.csv**: clusters popPUNK con nomenclatura específica del conjunto de datos

**poppunk_clusters_clusters_externos.csv**: Designaciones de esquemas GPSC v6

Podemos explorar "poppunk_clusters_clusters.csv"

>Nota: Si a una cepa ya se le ha asignado un cluster, por favor cambie el nombre para ejecutar popPUNK (esto es para evitar aplastar la herramienta) A los nuevos clusters se les asigna NA en el archivo _external_clusters.csv ya que no han sido definidos en el conjunto de datos v6 utilizado para designar los GPSCs. Por favor, envíe un correo electrónico a: globalpneumoseq@gmail.com para que se añadan nuevos clusters a la base de datos y se les asigne un nombre de cluster GPSC después de haber comprobado que no hay contaminación de bajo nivel que pueda contribuir a distancias accesorias sesgadas.

Este módulo popPUNK ha sido desarrollado a partir de las siguientes páginas web:

https://poppunk.readthedocs.io/en/latest/query_assignment.html

https://www.pneumogen.net/gps/training_command_line.html
